## Loess normalization on all features

#### Inspiration 

https://pypi.org/project/loess/

https://github.com/pharmbio/plaid/blob/main/simulations/PLAID-bioseminar-plots-2021.ipynb

https://github.com/pharmbio/plaid/blob/main/simulations/libraries/normalization.py 

In [1]:
import pandas as pd 
import numpy as np

import math, os

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("white")
from loess.loess_2d import loess_2d

pd.set_option('display.max_columns', 50)
#matplotlib inline
os.getcwd()


'/Users/chririn/Documents/Code/2D_analysis'

In [2]:
def front(self, n):
    return self.iloc[:, :n]

def back(self, n):
    return self.iloc[:, -n:]

pd.DataFrame.front = front
pd.DataFrame.back = back

# consider making this into a utility
# https://stackoverflow.com/questions/30608310/is-there-a-pandas-function-to-display-the-first-last-n-columns-as-in-head 

In [3]:
BaseDir = "results"
OutputDir = 'results'

if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)

## Load data 

In [4]:
df = pd.read_csv('{}/colo44_cleaned_data.csv'.format(BaseDir), sep=';' , index_col=0)
df.back(27).head(1)

,RadialDistribution_ZernikePhase_illumSYTO_9_9_cytoplasm,well_id,site,ImageID,barcode,plate_well,batch_id,solvent,stock_conc,stock_conc_unit,cmpd_vol,cmpd_vol_unit,well_vol,well_vol_unit,pert_type,cmpd_conc,cmpd_conc_unit,batchid,cbkid,libid,libtxt,smiles,inchi,inkey,conc,compound,Count_nuclei
0,0.426303,B02,1,colo44-v1-FA-P017365-CACO2-48h-P1-L1_B02_1,colo44-v1-FA-P017365-CACO2-48h-P1-L1,colo44-v1-FA-P017365-CACO2-48h-P1-L1_B02,PHB000015,dmso,10.0,mM,7.5,nL,30,uL,poscon,2.5,uM,PHB000015,[tetr],[tetr],pharmb_io,NaN,NaN,NaN,2.5,[tetr],74.0


In [6]:
df.front(27).head(1)

,ObjectNumber_nuclei,AreaShape_Area_nuclei,AreaShape_BoundingBoxArea_nuclei,AreaShape_BoundingBoxMaximum_X_nuclei,AreaShape_BoundingBoxMaximum_Y_nuclei,AreaShape_BoundingBoxMinimum_X_nuclei,AreaShape_BoundingBoxMinimum_Y_nuclei,AreaShape_Center_X_nuclei,AreaShape_Center_Y_nuclei,AreaShape_Compactness_nuclei,AreaShape_ConvexArea_nuclei,AreaShape_Eccentricity_nuclei,AreaShape_EquivalentDiameter_nuclei,AreaShape_EulerNumber_nuclei,AreaShape_Extent_nuclei,AreaShape_FormFactor_nuclei,AreaShape_MajorAxisLength_nuclei,AreaShape_MaxFeretDiameter_nuclei,AreaShape_MaximumRadius_nuclei,AreaShape_MeanRadius_nuclei,AreaShape_MedianRadius_nuclei,AreaShape_MinFeretDiameter_nuclei,AreaShape_MinorAxisLength_nuclei,AreaShape_Orientation_nuclei,AreaShape_Perimeter_nuclei,AreaShape_Solidity_nuclei,AreaShape_Zernike_0_0_nuclei
0,53.5,1860.0,2597.0,1062.5,806.5,1017.5,764.5,1037.132364,785.629139,1.201793,1933.5,0.696281,48.663968,1.0,0.725935,0.832093,57.710049,58.32857,20.383371,7.753648,7.0,42.026644,42.200039,-19.831706,170.145707,0.970882,0.690988


In [15]:
# Some information about the experiment
listOfPlates = df['barcode'].unique()
NrOfPlates = len(listOfPlates)
print('Number of plates: %s' %(NrOfPlates))
Wells = sorted(list(set(df['well_id'])))
NrOfWells = len(Wells)
print('Number of wells: %s' %(NrOfWells))
Rows = sorted(list(set([w[0] for w in Wells])))
print(*Rows)
NrOfRows = len(Rows)
Columns = sorted(list(set([w[1:] for w in Wells])))
NrOfColumns = len(Columns)
print(*Columns)
ListOfFeatures = df.columns[:-26].to_list()## Select all the features.
NrOfFeatures = len(ListOfFeatures)
print('Number of plates: %s' %(NrOfFeatures))

Number of plates: 4
Number of wells: 308
B C D E F G H I J K L M N O
02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23
Number of plates: 2182


In [ ]:
for feature in ListOfFeatures:
    OneFeature = df[[feature, 'well_id', 'barcode', 'compound']]
    OneFeatureDMSO = OneFeature.loc[(OneFeature['compound'] == '[dmso]'), [feature, 'well_id', 'barcode', 'compound']]

    for OnePlate in listOfPlates:

In [ ]:
for feature in FeaturesToPlot[:]:
    fig = plt.figure(figsize = HeatmapSize) # Check if I can optimize the size for my purpose
    fig.suptitle('{}'.format(feature) + " Loess norm", fontsize = 16, y=1.1) 

    OneFeature = df[[feature, 'well_id', 'barcode', 'compound']]
    OneFeatureDMSO = OneFeature.loc[(OneFeature['compound'] == '[dmso]'), [feature, 'well_id', 'barcode', 'compound']]

    # Set up the heatmap boundaries
    vmed = math.ceil(OneFeature[feature].quantile(0.5)*1000)/1000
    vmed_DMSO = math.ceil(OneFeatureDMSO[feature].quantile(0.5)*1000)/1000
    vmin = 0 

    n = 1 # subplot counter

    for OnePlate in listOfPlates:
        OneFeaturePerPlate = OneFeature[OneFeature['barcode'] == OnePlate]
        OneFeaturePerPlateDMSO = OneFeatureDMSO[OneFeatureDMSO['barcode'] == OnePlate]

        dictOfnumeric = {feature : np.median} 
        groupedby = OneFeaturePerPlate.groupby(['well_id']).agg(dictOfnumeric) # Aggregate to one value per well
        groupedbyDMSO = OneFeaturePerPlateDMSO.groupby(['well_id']).agg(dictOfnumeric)

        # Apply the loess normalization
        # Prepare the data: DMSO 
        groupedbyDMSO = OneFeaturePerPlateDMSO.groupby(['well_id']).agg(dictOfnumeric)
        groupedbyDMSO["columns"] = groupedbyDMSO.index.str[:1]
        groupedbyDMSO["columns"] = [ ord(x) - 64 for x in groupedbyDMSO["columns"]]
        groupedbyDMSO["rows"] = groupedbyDMSO.index.str[1:3].str.lstrip('0')
        groupedbyDMSO["rows"] = groupedbyDMSO["rows"].astype(int)

        x = groupedbyDMSO[["columns"]].to_numpy().reshape((-1,))
        y = groupedbyDMSO[["rows"]].to_numpy().reshape((-1,))
        z = groupedbyDMSO[feature].to_numpy().reshape((-1,))

        # Prepare the data: all wells 
        groupedby = OneFeaturePerPlate.groupby(['well_id']).agg(dictOfnumeric) # Aggregate to one value per well
        groupedby["columns"] = groupedby.index.str[:1]
        groupedby["columns"] = [ ord(x) - 64 for x in groupedby["columns"]]
        groupedby["rows"] = groupedby.index.str[1:3].str.lstrip('0')
        groupedby["rows"] = groupedby["rows"].astype(int)

        xnew = groupedby[["columns"]].to_numpy().reshape((-1,))
        ynew = groupedby[["rows"]].to_numpy().reshape((-1,))
        znew = groupedby[feature].to_numpy().reshape((-1,))

        # The actual normalization 
        zout, _ = loess_2d(x, y, z, xnew=xnew, ynew=ynew, degree=1, frac=1.0, npoints=None, rescale=False, sigz=None) # estimations of the curves 
        zout_controls, _ = loess_2d(x, y, z, degree=1, frac=1.0, npoints=None, rescale=False, sigz=None)
        z_norm = znew - zout + np.nanmean(zout_controls) # calculation of the normalized data
        
        groupedby["z_norm"] = z_norm

        # Remove non-exisiting wells
        for W in Wells:
            if W not in groupedby.index:
                groupedby.loc[W] = 0
        groupedby.sort_index(inplace=True) 

        ## OBS: here is the problem! 

        # Prepare data for plotting
        groupedbyvalues = groupedby["z_norm"].values.copy()
        groupedbyvalues = groupedbyvalues.reshape(NrOfRows,int(NrOfColumns))
        groupedby_z_norm = pd.DataFrame(groupedbyvalues)
        groupedby_z_norm.columns = Columns
        groupedby_z_norm.index = Rows

        # Subplots 
        ax = fig.add_subplot(Ro, Co , n)
        ax = sns.heatmap(groupedby_z_norm, cmap=cmap, # all data
                                     vmin = vmin,
                                     vmax = vmed * 2.5,
                                     square = False,
                                     linewidths = .5,
                                     xticklabels = 1, yticklabels = groupedby_z_norm.index,
                                     )

        plt.yticks(fontsize=7,rotation=0)
        plt.xticks(fontsize=8,rotation= -45) 

        ax.set_title(OnePlate, fontsize = 12, loc = 'center')
        plt.subplots_adjust(  wspace = wspace, hspace = hspace, top = top)
        n += 1